In [1]:
!nvidia-smi

Wed Sep 27 09:10:26 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     Off | 00000000:AF:00.0 Off |                    0 |
|  0%   30C    P0              69W / 300W |  15303MiB / 46068MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document

In [4]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})

/usr/local/lib/python3.10/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer


2023-09-27 09:11:09.012615: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-27 09:11:09.904159: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


max_seq_length  512


In [5]:
persist_directory = 'db'

vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=instructor_embeddings)

In [10]:
import yaml
# https://github.com/gunthercox/chatterbot-corpus/blob/master/chatterbot_corpus/data/english/greetings.yml

In [12]:
with open('./data/english/greetings.yml', 'r') as file:
    greetings = yaml.safe_load(file)

In [13]:
greetings

{'categories': ['greetings'],
 'conversations': [['Hello', 'Hi'],
  ['Hi', 'Hello'],
  ['Greetings!', 'Hello'],
  ['Hello', 'Greetings!'],
  ['Hi, How is it going?', 'Good'],
  ['Hi, How is it going?', 'Fine'],
  ['Hi, How is it going?', 'Okay'],
  ['Hi, How is it going?', 'Great'],
  ['Hi, How is it going?', 'Could be better.'],
  ['Hi, How is it going?', 'Not so great.'],
  ['How are you doing?', 'Good.'],
  ['How are you doing?', 'Very well, thanks.'],
  ['How are you doing?', 'Fine, and you?'],
  ['Nice to meet you.', 'Thank you.'],
  ['How do you do?', "I'm doing well."],
  ['How do you do?', "I'm doing well. How are you?"],
  ['Hi, nice to meet you.', 'Thank you. You too.'],
  ['It is a pleasure to meet you.', 'Thank you. You too.'],
  ['Top of the morning to you!', 'Thank you kindly.'],
  ['Top of the morning to you!', 'And the rest of the day to you.'],
  ["What's up?", 'Not much.'],
  ["What's up?", 'Not too much.'],
  ["What's up?", 'Not much, how about you?'],
  ["What's up?

In [28]:
conversations = list(map(lambda g: " ".join(g), greetings["conversations"]))

In [32]:
greeting_docs = list(map(lambda x: Document(page_content=x, metadata={"category": greetings["categories"][0]}), conversations))

In [33]:
greeting_docs[0]

Document(page_content='Hello Hi', metadata={'category': 'greetings'})

In [34]:
vectordb._collection.count()

6

In [40]:
new_ids = [greetings["categories"][0]+"_"+str(i) for i in range(len(greeting_docs))]

In [41]:
return_ids = vectordb.add_documents(
    greeting_docs,
    ids=new_ids
)

In [43]:
vectordb._collection.count()

31

In [55]:
vectordb._collection.get(ids=["greetings_0","greetings_1"])

{'ids': ['greetings_0', 'greetings_1'],
 'embeddings': None,
 'documents': ['Hello Hi', 'Hi Hello'],
 'metadatas': [{'category': 'greetings'}, {'category': 'greetings'}]}

vectordb.get(
    where={"category": greetings["categories"][0]}
)

### Cosine distance the lower the better [0,1]

### 相似問候語

In [76]:
# 相似問候語
vectordb.similarity_search_with_score("How are you?", k = 3)

[(Document(page_content='How are you doing? Fine, and you?', metadata={'category': 'greetings'}),
  0.23643285036087036),
 (Document(page_content="How do you do? I'm doing well. How are you?", metadata={'category': 'greetings'}),
  0.2597610354423523),
 (Document(page_content="How do you do? I'm doing well.", metadata={'category': 'greetings'}),
  0.29596203565597534)]

### 非數據庫問候語

In [75]:
# 非數據庫問候語
vectordb.similarity_search_with_score("Good night!", k = 3)

[(Document(page_content='Top of the morning to you! And the rest of the day to you.', metadata={'category': 'greetings'}),
  0.8002604842185974),
 (Document(page_content='Top of the morning to you! Thank you kindly.', metadata={'category': 'greetings'}),
  0.82635498046875),
 (Document(page_content='How are you doing? Good.', metadata={'category': 'greetings'}),
  0.914035439491272)]

### 非問候語

In [74]:
# 非問候語
vectordb.similarity_search_with_score("How can I navigate to the RMA request page in the Wareconn Customer Portal?", k = 10)

[(Document(page_content='2 / 6 \nThis document  belongs to Wareconn  Technology Services (Tianjin) Co., Ltd. It is only intended to be \nused to introduce Wareconn functions and procedures. Please do not use it for other purposes.  2 wareconn standard operating procedure  \nSOP  \nI. This guide will walk you through the process of requesting  RMA  through the \nWareconn Customer Portal.  \nII. Details on how to fill in the Part Number (PN) and Serial Number (SN)  \n1.1 RMA request  procedure (Parts return)  \n1. System login  \n⚫ Module: https://www.wareconn.com/  \n⚫ Description:  Login  with account and password  \n \nTo log in, follow these steps:  \nGo to https://www.wareconn.com/ and click Log in  \n \nFill in your email and password, then click Login\n \nClick Enter  in the Customer portal', metadata={'source': 'MSFT RMA request SOP v1.0.pdf', 'page': 1}),
  0.26276934146881104),
 (Document(page_content='3 / 6 \nThis document  belongs to Wareconn  Technology Services (Tianjin) Co

### 完全相同問候語

In [73]:
# 完全相同問候語
vectordb.similarity_search_with_score("How are you doing? Fine, and you?", k = vectordb._collection.count())

[(Document(page_content='How are you doing? Fine, and you?', metadata={'category': 'greetings'}),
  0.0259263776242733),
 (Document(page_content="How do you do? I'm doing well. How are you?", metadata={'category': 'greetings'}),
  0.14784325659275055),
 (Document(page_content='How are you doing? Good.', metadata={'category': 'greetings'}),
  0.17279070615768433),
 (Document(page_content="How do you do? I'm doing well.", metadata={'category': 'greetings'}),
  0.18660202622413635),
 (Document(page_content='How are you doing? Very well, thanks.', metadata={'category': 'greetings'}),
  0.19339334964752197),
 (Document(page_content='Hi, How is it going? Fine', metadata={'category': 'greetings'}),
  0.3499649465084076),
 (Document(page_content="What's up? The sky's up but I'm fine thanks. What about you?", metadata={'category': 'greetings'}),
  0.4036705195903778),
 (Document(page_content='Hi, How is it going? Great', metadata={'category': 'greetings'}),
  0.4600520730018616),
 (Document(pag